In [62]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import os.path  # To manage paths
import sys
import time 
from datetime import datetime


import backtrader as bt
import backtrader.analyzers as btanalyzers
import backtrader.strategies as btstrats

In [65]:
import sys
import os
sys.path.append(os.path.abspath("../.."))


import pandas as pd
import numpy as np

# Read the CSV file
df_raw = pd.read_csv('../../data/processed-data/g_2010_2023.csv', index_col="Date", parse_dates=["Date"])
print(os.getcwd())
print(df_raw.head())
import joblib

scaler = joblib.load('./scaler.pkl')



from scripts.process import process

X, Y = process(df_raw, scaler, dev=False)
print(X.shape, Y.shape)
print(scaler)

/home/elm19/Desktop/GoldSpot-Predictor/notebooks/new_approach_exp1
             close    open    high     low    volume
Date                                                
2010-01-04  1118.3  1099.0  1124.6  1093.8  155480.0
2010-01-05  1118.7  1122.0  1129.6  1115.5  156410.0
2010-01-06  1136.5  1118.5  1141.0  1116.8  176900.0
2010-01-07  1133.7  1139.0  1139.5  1128.7  121150.0
2010-01-08  1138.9  1131.5  1140.0  1119.5  212620.0
(3855, 20, 12) (3855, 3)
StandardScaler()


In [ ]:
# #!/usr/bin/env python3

# import numpy as np
# from keras.models import load_model

# # 1) Load your saved model
# model = load_model('./regulized_model.keras')

# # 2) Prepare your input data as a NumPy array of shape (n_samples, seq_len, n_features)
# #    For example:
# # X_new = np.load('X_new.npy')

# # 3) Get class‐probabilities and class indices
# proba   = model.predict(X)             
# pred_ix = np.argmax(proba, axis=1)         

# # 4) (Optional) Map back to labels
# class_map = {0: 'sell', 1: 'hold', 2: 'buy'}
# pred_labels = [class_map[i] for i in pred_ix]


121/121 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step
float32


In [ ]:
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model.evaluate(X, Y, verbose=0)

[0.5414470434188843, 0.7553825974464417]

In [25]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    params = dict(
        atr_period=14,
        atr_multiplier=1.5,  # for stop-loss
        risk_per_trade = 0.01
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.atr = bt.ind.ATR(self.data, period=self.p.atr_period)
        self.order = None
        
        


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status == order.Canceled:
            print("Order was canceled.")

        elif order.status == order.Margin:
            print("Order was rejected due to insufficient margin.")

        elif order.status == order.Rejected:
            print("Order was rejected by the broker.")

        # Write down: no pending order
        self.order = None

    def next(self):
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        
        if not self.position and len(self) >= 34:
            print(self.datas[0].close[0])
            # self.log(self.dataclose)
            pass
        
        # time.sleep(1)

    def place_order(self, order_type, atr_value):
        stop_loss_distance = atr_value * self.p.atr_multiplier

        capital = self.broker.getvalue()
        risk_amount = capital * self.p.risk_per_trade
        size = risk_amount / stop_loss_distance
        self.price = self.data.close[0]
        
        if order_type == "buy":
            self.stop_price = self.price - stop_loss_distance
            # self.buy(size=size,  trailamount=stop_loss_distance)

            self.buy_bracket(
                price=self.price,
                size=size,
                # exectype=bt.Order.TrailingStop,
                # trailamount=stop_loss_distance,
                stopprice=self.stop_price,
                limitprice=self.price + stop_loss_distance,
            )
        elif order_type=="sell":
            self.stop_price = self.price + stop_loss_distance
            # self.sell(size=size,  trailamount=stop_loss_distance)
            self.sell_bracket(
                price=self.price,
                size=size,
                limitprice=self.price - stop_loss_distance,
                stopprice=self.stop_price
            )
        

In [31]:
# # Create a cerebro entity
# cerebro = bt.Cerebro()
# # Add a strategy
# cerebro.addstrategy(TestStrategy)
# cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')
# cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
# cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
# # cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
# cerebro.addanalyzer(bt.analyzers.SQN, _name='sqn')
# cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')

# # Datas are in a subfolder of the samples. Need to find where the script is
# # because it could have been called from anywhere
# modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
# datapath = '../../data/processed-data/g_2010_2023.csv'

# # Create a Data Feed# Add a strategy
# data = bt.feeds.GenericCSVData(
#     dataname=datapath,
#     dtformat=('%Y-%m-%d'),
#     datetime=0,
#     time=-1,
#     openinterest=-1,
#     high=3,
#     low=4,
#     close=1,
#     volume=5,
#     open=2,
#     # nullvalue=0.0
# ) # type: ignore
# # Add the Data Feed to Cerebro
# cerebro.adddata(data)

# # Set our desired cash start
# cerebro.broker.setcash(10000.0)

# # Print out the starting conditions
# print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# # Run over everything
# strats = cerebro.run()
# # cerebro.plot()
# # strat = strats[0]


# # print("Total Trades:", strat.analyzers.trades.get_analysis()['total']['total'])
# # print("Net Profit:", strat.analyzers.trades.get_analysis()['pnl']['net']['total'])
# # print("Sharpe Ratio:", strat.analyzers.sharpe.get_analysis())
# # print("Max Drawdown %:", strat.analyzers.drawdown.get_analysis()['max']['drawdown'])
# # print("Max Drawdown Value:", strat.analyzers.drawdown.get_analysis()['max']['moneydown'])
# # print("SQN:", strat.analyzers.sqn.get_analysis())
# # print("Returns:", strat.analyzers.returns.get_analysis()['rtot'])

# # pyfoliozer = strat.analyzers.getbyname('pyfolio')
# # returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()


# # Print out the final result
# print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

In [ ]:
import sys
import os
sys.path.append(os.path.abspath("../.."))

import pandas as pd
import numpy as np

# Read the CSV file
df_raw = pd.read_csv('../../data/processed-data/g_2010_2023.csv', index_col="Date", parse_dates=["Date"])
print(os.getcwd())
print(df_raw.head())
import joblib

scaler = joblib.load('./scaler.pkl')



from scripts.process import process

X, Y = process(df_raw, scaler, dev=False)


In [215]:
import numpy as np
import pandas as pd
from keras.models import load_model

class ModelStrategy(bt.Strategy):
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
    def __init__(self):
        self.indicators = {}
        data = self.datas[0]  # shorthand
        self.signalProviderModel = load_model(model['path'])
        for indicator in model['indicators']:
            name = indicator['name']
            if name == 'sma':
                self.indicators['SMA_20'] = bt.indicators.SimpleMovingAverage(data.close, period=indicator['period']) # type: ignore
            elif name == 'rsi':
                self.indicators['RSI'] = bt.indicators.RSI(data.close, period=indicator['period']) # type: ignore
            elif name == 'atr':
                self.indicators['ATR'] = bt.indicators.ATR(data, period=indicator['period']) # type: ignore
            elif name == 'macd':
                self.indicators['MACD'] = bt.indicators.MACD(data.close,
                                                             period_me1=indicator['fast'],
                                                             period_me2=indicator['slow'],
                                                             period_signal=indicator['signal'])
        self.target_indicator = [np.nan] * 7  # Initialize with NaN for the first 7 values
        

    def next(self):
        n = model['sequence_length'] + model['max_data_ind'] 
        if len(self.datas[0]) < n -1:  # Ensure we have at least 20 days of data
            return

        dt = self.datas[0].datetime.date(0)
        

        
        interval = reversed(range(-1, n-1))
        # Collect data for the previous 20 days
        data_dict = {
            'Date': [self.datas[0].datetime.date(-i) for i in interval],
            'close': [self.datas[0].close[-i] for i in reversed(range(-1, n-1))],
            'open': [self.datas[0].open[-i] for i in reversed(range(-1, n-1))],
            'high': [self.datas[0].high[-i] for i in reversed(range(-1, n-1))],
            'low': [self.datas[0].low[-i] for i in reversed(range(-1, n-1))],
            'volume': [self.datas[0].volume[-i] for i in reversed(range(-1, n-1))],
            # 'SMA_20': [self.indicators['SMA_20'][-i] for i in reversed(range(20))],
            # 'RSI': [self.indicators['RSI'][-i] for i in reversed(range(20))],
            # 'ATR': [self.indicators['ATR'][-i] for i in reversed(range(20))],
            # 'MACD_12_26_9': [self.indicators['MACD'].macd[-i] for i in reversed(range(20))],
            # 'MACDh_12_26_9': [self.indicators['MACD'].signal[-i] for i in reversed(range(20))],
            # 'MACDs_12_26_9': [self.indicators['MACD'].macd[-i] - self.indicators['MACD'].signal[-i] for i in reversed(range(20))],
            # 'Target': (self.target_indicator[-20:] + [np.nan] * (20 - len(self.target_indicator[-20:])))[:20],
        }
        index = data_dict.pop('Date')  # remove 'Date' from columns and keep as index
        df = pd.DataFrame(data_dict, index=index)
        df.index.name = 'Date'
        # scaled = scaler.transform(df)
        # scaled_df = pd.DataFrame(scaled, columns=df.columns, index=df.index)
        # scaled_df['Target'] = self.target_indicator[-20:] + [np.nan] * (20 - len(self.target_indicator[-20:]))
        X, y = process(df, scaler, dev=False)
        
        proba   = self.signalProviderModel.predict(X)             
        pred_ix = np.argmax(proba, axis=1)  
        # Create a DataFrame
        class_map = {0: 'sell', 1: 'hold', 2: 'buy'}
        if pred_ix ==0:
            self.log('sell')
        if pred_ix ==2:
            self.log('buy')

In [216]:
# Create a cerebro entity
cerebro = bt.Cerebro()
# Add a strategy
cerebro.addstrategy(ModelStrategy)

modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
datapath = '../../data/processed-data/g_2010_2023.csv'

# Create a Data Feed# Add a strategy
data = bt.feeds.GenericCSVData(
    dataname=datapath,
    dtformat=('%Y-%m-%d'),
    datetime=0,
    time=-1,
    openinterest=-1,
    high=3,
    low=4,
    close=1,
    volume=5,
    open=2,
    # nullvalue=0.0
) # type: ignore
# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(10000.0)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
strats = cerebro.run()


# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 781ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━

2025-06-08 19:46:00.258069: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/home/elm19/miniconda3/envs/goldspot-predictor/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


ValueError: math domain error

In [ ]:
df_raw = pd.read_csv('../../data/processed-data/g_2010_2023.csv', index_col="Date", parse_dates=["Date"])
print(os.getcwd())
print(df_raw.head())

/home/elm19/Desktop/GoldSpot-Predictor/notebooks/new_approach_exp1
             close    open    high     low    volume
Date                                                
2010-01-04  1118.3  1099.0  1124.6  1093.8  155480.0
2010-01-05  1118.7  1122.0  1129.6  1115.5  156410.0
2010-01-06  1136.5  1118.5  1141.0  1116.8  176900.0
2010-01-07  1133.7  1139.0  1139.5  1128.7  121150.0
2010-01-08  1138.9  1131.5  1140.0  1119.5  212620.0


Running the model to predict for each day takes a lot of time, we will try to run the predictions outside of backtrader and then use them inside backtrader for the backtest

In [222]:
# first import the data again 
df_raw = pd.read_csv('../../data/processed-data/g_2010_2023.csv', index_col="Date", parse_dates=["Date"])
print(df_raw.head())


import joblib
scaler = joblib.load('./scaler.pkl')
from scripts.process import process
from collections import Counter
X, Y = process(df_raw, scaler, dev=False)

# 1) Load your saved model
model = load_model('./regulized_model.keras')
proba   = model.predict(X)             
pred_ix = np.argmax(proba, axis=1)
class_map = {0: 'sell', 1: 'hold', 2: 'buy'}
pred_labels = [class_map[i] for i in pred_ix]


# lets count how many buy signals we got 
counts = Counter(pred_labels)
print("Counts:", counts)

             close    open    high     low    volume
Date                                                
2010-01-04  1118.3  1099.0  1124.6  1093.8  155480.0
2010-01-05  1118.7  1122.0  1129.6  1115.5  156410.0
2010-01-06  1136.5  1118.5  1141.0  1116.8  176900.0
2010-01-07  1133.7  1139.0  1139.5  1128.7  121150.0
2010-01-08  1138.9  1131.5  1140.0  1119.5  212620.0
121/121 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Counts: Counter({'hold': 1870, 'buy': 1115, 'sell': 870})


In [308]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    params = dict(
        atr_period=14,
        atr_multiplier=1.0,  # for trailing stop-loss
        risk_per_trade=0.01
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt), self.broker.getvalue())

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.atr = bt.ind.ATR(self.data, period=self.p.atr_period)
        self.order = None
        self.position_type = None  # Track the type of position: 'buy' or 'sell'
        
        


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
                self.position_type = 'buy'
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)
                self.position_type = 'sell'
        
            self.bar_executed = len(self)

        elif order.status == order.Canceled:
            print("Order was canceled.")

        elif order.status == order.Margin:
            print("Order was rejected due to insufficient margin.")

        elif order.status == order.Rejected:
            print("Order was rejected by the broker.")

        # Write down: no pending order
        self.order = None

    def next(self):
        n = 34 + 20 + 1  # 34 days of data + 20 days of prediction
        if len(self) <= n or len(self)>len(pred_labels) + n:  # Ensure we have enough data
            return
        print(len(self), len(pred_labels))
        prediction = pred_labels[len(self) - n-1]  # Get the model prediction for the current day

        if not self.position:
            # If not in a position, place a buy or sell order based on the model prediction
            if prediction == 'buy':
                self.place_order('buy')
            elif prediction == 'sell':
                self.place_order('sell')
        else:
            # If in a position, check if the model prediction matches the current position
            if self.position_type == 'buy' and prediction == 'sell':
                self.log('EXIT BUY AND ENTER SELL')
                self.close()
                self.place_order('sell')
            elif self.position_type == 'sell' and prediction == 'buy':
                self.log('EXIT SELL AND ENTER BUY')
                self.close()
                self.place_order('buy')
            # If the prediction matches the current position, do nothing

    def place_order(self, order_type):
        stop_loss_distance = self.atr[0] * self.p.atr_multiplier

        capital = self.broker.getvalue()
        risk_amount = capital * self.p.risk_per_trade
        size = risk_amount / stop_loss_distance
        price = self.data.close[0]
        
        if order_type == 'buy':
            stop_price = price - stop_loss_distance
            self.buy(size=size, exectype=bt.Order.StopTrail, trailamount=stop_loss_distance)
        elif order_type == 'sell':
            stop_price = price + stop_loss_distance
            self.sell(size=size, exectype=bt.Order.StopTrail, trailamount=stop_loss_distance)

In [310]:
# Create a cerebro entity
cerebro = bt.Cerebro()
# Add a strategy
cerebro.addstrategy(TestStrategy)
cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name='trades')
cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='sharpe')
cerebro.addanalyzer(btanalyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(btanalyzers.SQN, _name='sqn')
cerebro.addanalyzer(btanalyzers.Returns, _name='returns')

# Datas are in a subfolder of the samples. Need to find where the script is
# because it could have been called from anywhere
modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
datapath = '../../data/processed-data/g_2010_2023.csv'

# Create a Data Feed# Add a strategy
import backtrader.feeds as btfeeds

data = btfeeds.GenericCSVData(
    dataname=datapath,
    dtformat=('%Y-%m-%d'),
    
    datetime=0,
    time=-1,
    openinterest=-1,
    high=3,
    low=4,
    close=1,
    volume=5,
    open=2,
    # nullvalue=0.0
) # type: ignore
# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(10000.0)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
strats = cerebro.run()
cerebro.plot()
strat = strats[0]


print("Total Trades:", strat.analyzers.trades.get_analysis()['total']['total'])
print("Net Profit:", strat.analyzers.trades.get_analysis()['pnl']['net']['total'])
print("Sharpe Ratio:", strat.analyzers.sharpe.get_analysis())
print("Max Drawdown %:", strat.analyzers.drawdown.get_analysis()['max']['drawdown'])
print("Max Drawdown Value:", strat.analyzers.drawdown.get_analysis()['max']['moneydown'])
print("SQN:", strat.analyzers.sqn.get_analysis())
print("Returns:", strat.analyzers.returns.get_analysis()['rtot'])


# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
56 3855
57 3855
58 3855
59 3855
60 3855
61 3855
2010-04-01, BUY EXECUTED, 1123.73 10013.16168211417
Order was rejected due to insufficient margin.
62 3855
Order was rejected due to insufficient margin.
63 3855
64 3855
65 3855
66 3855
67 3855
68 3855
69 3855
70 3855
2010-04-14, EXIT BUY AND ENTER SELL 10198.990914997736
2010-04-15, SELL EXECUTED, 1155.80 10177.91177813333
71 3855
2010-04-16, SELL EXECUTED, 1143.94 10268.600430520684
2010-04-16, SELL EXECUTED, 1144.28 10268.600430520684
72 3855
73 3855
74 3855
75 3855
76 3855
2010-04-22, EXIT SELL AND ENTER BUY 10193.098461464617
2010-04-23, BUY EXECUTED, 1141.80 10206.940489124896
77 3855
2010-04-26, BUY EXECUTED, 1159.36 10173.746398472093
78 3855
Order was rejected due to insufficient margin.
79 3855
80 3855
81 3855
82 3855
83 3855
84 3855
85 3855
86 3855
87 3855
88 3855
89 3855
90 3855
91 3855
92 3855
2010-05-14, EXIT BUY AND ENTER SELL 10630.7538556238
2010-05-17, SELL EXECUTED, 1232.00 10656.76241

<IPython.core.display.Javascript object>

Total Trades: 244
Net Profit: 8023735.525744609
Sharpe Ratio: OrderedDict([('sharperatio', 3.2365986616259326)])
Max Drawdown %: 9.357143066250659
Max Drawdown Value: 187982.63625798933
SQN: AutoOrderedDict([('sqn', 5.739994406440729), ('trades', 243)])
Returns: 6.709855681246686
Final Portfolio Value: 8204522.24
